In [1]:
import json
import pandas as pd

In [2]:
list_model = [50,60,33,57,36,55,31,54,71,80,53,60,83,60]
print(len(list_model))
data_merge = []

for i in range(len(list_model)):
    class_id = i
    with open('/media/sonnh/kaggle-vin/runs/test_train_19/last{}_predictions.json'.format(list_model[i]), 'r') as dir:
        data = json.load(dir)
        for anno in data:
            if anno['category_id'] == class_id:
                data_merge.append(anno)

14


In [3]:
len(data_merge)

334469

In [4]:
with open('/media/sonnh/kaggle-vin/runs/test_train_19/merge_predictions.json', 'w') as dir:
    json.dump(data_merge, dir)

In [18]:
 with open('/root/annotations/instances_val2017.json', 'r') as dir:
    data_coco = json.load(dir)



In [19]:
data_coco.keys()


dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])

In [59]:
data['categories']

[{'id': 1, 'name': 'person', 'supercategory': 'person'},
 {'id': 2, 'name': 'bicycle', 'supercategory': 'vehicle'},
 {'id': 3, 'name': 'car', 'supercategory': 'vehicle'},
 {'id': 4, 'name': 'motorcycle', 'supercategory': 'vehicle'},
 {'id': 5, 'name': 'airplane', 'supercategory': 'vehicle'},
 {'id': 6, 'name': 'bus', 'supercategory': 'vehicle'},
 {'id': 7, 'name': 'train', 'supercategory': 'vehicle'},
 {'id': 8, 'name': 'truck', 'supercategory': 'vehicle'},
 {'id': 9, 'name': 'boat', 'supercategory': 'vehicle'},
 {'id': 10, 'name': 'traffic light', 'supercategory': 'outdoor'},
 {'id': 11, 'name': 'fire hydrant', 'supercategory': 'outdoor'},
 {'id': 13, 'name': 'stop sign', 'supercategory': 'outdoor'},
 {'id': 14, 'name': 'parking meter', 'supercategory': 'outdoor'},
 {'id': 15, 'name': 'bench', 'supercategory': 'outdoor'},
 {'id': 16, 'name': 'bird', 'supercategory': 'animal'},
 {'id': 17, 'name': 'cat', 'supercategory': 'animal'},
 {'id': 18, 'name': 'dog', 'supercategory': 'animal'},

# gt to coco

In [17]:
df = pd.read_csv('/root/train_only_box_merge_box_0.4.csv')
df = df[df['fold'] == 0]
df.head()

,image_id,class_id,x_min,y_min,x_max,y_max,fold
15,5bf3368744630f459a499ccdccc9cdf1,0,1437.0,933.3,1833.3,1413.7,0
16,5bf3368744630f459a499ccdccc9cdf1,3,1065.0,2005.5,2343.5,2584.0,0
17,5bf3368744630f459a499ccdccc9cdf1,9,960.0,427.0,1193.0,671.0,0
18,5bf3368744630f459a499ccdccc9cdf1,9,1974.0,253.0,2148.0,479.0,0
19,5bf3368744630f459a499ccdccc9cdf1,6,468.0,1826.0,1133.0,2194.0,0


In [64]:
gt_data = {'info':[], 'licenses':[], 'images':[], 'annotations':[], 'categories':[]}
list_image_id = []
count_box = 0
for i in range(len(df)):
    image_id = df.iloc[i]['image_id']
    x_min = int(df.iloc[i]['x_min'])
    y_min = int(df.iloc[i]['y_min'])
    x_max = int(df.iloc[i]['x_max'])
    y_max = int(df.iloc[i]['y_max'])
    w = x_max - x_min
    h = y_max - y_min
    class_id = int(df.iloc[i]['class_id'])
    
    if image_id not in list_image_id:
        list_image_id.append(image_id)
        
        gt_data['images'].append({
            'id': image_id,
            "height": 626,
            "width": 1622,})
    gt_data['annotations'].append({'area': w*h,
                                 'bbox': [x_min, y_min, w, h],
                                 'category_id': class_id,
                                 'id': count_box,
                                 'image_id': image_id,
                                 'iscrowd': 0,})
    count_box += 1
        
    

In [65]:
with open('/root/kaggle-VinBigData-Chest-X-ray-Abnormalities-Detection/14_class_predict_val/gt_coco_format.json', 'w') as dir:
    json.dump(gt_data, dir)

In [66]:
anno_json = '/root/kaggle-VinBigData-Chest-X-ray-Abnormalities-Detection/14_class_predict_val/gt_coco_format.json'  # annotations json
pred_json = '/root/kaggle-VinBigData-Chest-X-ray-Abnormalities-Detection/14_class_predict_val/merge_predictions.json'

# try:  # https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoEvalDemo.ipynb
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

anno = COCO(anno_json)  # init annotations api
pred = anno.loadRes(pred_json)  # init predictions api
eval = COCOeval(anno, pred, 'bbox')


eval.params.imgIds = [gt_data['images'][i]['id'] for i in range(len(gt_data['images']))]  # image IDs to evaluate
    
eval.evaluate()
eval.accumulate()
eval.summarize()


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=1.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.15s).
Accumulating evaluation results...
Please run evaluate() first
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = -1.000
 Average Recall     (AR) @[ I

# check

In [45]:
with open(pred_json, 'r') as dir:
    data_pred = json.load(dir)

In [52]:
for anno in data_pred:
    if anno['image_id'] == '852ec6a0bcb9608ffee9b3fda0867f91':
        print(anno)

{'image_id': '852ec6a0bcb9608ffee9b3fda0867f91', 'category_id': 0, 'bbox': [1636.407, 873.558, 345.964, 415.156], 'score': 0.65723}
{'image_id': '852ec6a0bcb9608ffee9b3fda0867f91', 'category_id': 0, 'bbox': [1195.304, 1270.551, 309.637, 365.856], 'score': 0.08081}
{'image_id': '852ec6a0bcb9608ffee9b3fda0867f91', 'category_id': 0, 'bbox': [1762.684, 1233.36, 318.286, 364.991], 'score': 0.03093}
{'image_id': '852ec6a0bcb9608ffee9b3fda0867f91', 'category_id': 0, 'bbox': [1350.988, 882.207, 610.626, 527.594], 'score': 0.02036}
{'image_id': '852ec6a0bcb9608ffee9b3fda0867f91', 'category_id': 0, 'bbox': [1612.19, 1314.661, 411.697, 518.945], 'score': 0.01226}
{'image_id': '852ec6a0bcb9608ffee9b3fda0867f91', 'category_id': 0, 'bbox': [1318.121, 1024.917, 363.262, 486.944], 'score': 0.00697}
{'image_id': '852ec6a0bcb9608ffee9b3fda0867f91', 'category_id': 0, 'bbox': [1263.632, 1811.119, 1023.187, 356.342], 'score': 0.00435}
{'image_id': '852ec6a0bcb9608ffee9b3fda0867f91', 'category_id': 0, 'bbox

In [48]:
with open(anno_json, 'r') as dir:
    data_anno = json.load(dir)

In [51]:
for anno in data_anno['annotations']:
    if anno['image_id'] == '852ec6a0bcb9608ffee9b3fda0867f91':
        print(anno)

{'area': 164102.39999999997, 'bbox': [1607.7, 887.0, 369.5999999999999, 444.0], 'category_id': 0, 'id': 2346, 'image_id': '852ec6a0bcb9608ffee9b3fda0867f91', 'iscrowd': 0}
{'area': 360336.21999999986, 'bbox': [1271.7, 1864.3, 1031.3, 349.39999999999986], 'category_id': 3, 'id': 2347, 'image_id': '852ec6a0bcb9608ffee9b3fda0867f91', 'iscrowd': 0}
{'area': 141602.25, 'bbox': [1424.0, 228.5, 261.5, 541.5], 'category_id': 9, 'id': 2348, 'image_id': '852ec6a0bcb9608ffee9b3fda0867f91', 'iscrowd': 0}
{'area': 298116.0, 'bbox': [1149.0, 220.0, 468.0, 637.0], 'category_id': 9, 'id': 2349, 'image_id': '852ec6a0bcb9608ffee9b3fda0867f91', 'iscrowd': 0}
{'area': 46440.0, 'bbox': [1040.0, 773.0, 270.0, 172.0], 'category_id': 13, 'id': 2350, 'image_id': '852ec6a0bcb9608ffee9b3fda0867f91', 'iscrowd': 0}
{'area': 85500.0, 'bbox': [1176.0, 960.0, 228.0, 375.0], 'category_id': 13, 'id': 2351, 'image_id': '852ec6a0bcb9608ffee9b3fda0867f91', 'iscrowd': 0}
{'area': 119637.0, 'bbox': [1451.0, 130.0, 189.0, 63